# Working with NBA live data...
Libraries supporting live data do not include support for Pandas. Note: Any call to `{endpoint}.{Class}()` will perform a request. Example: `scoreboard.ScoreBoard()`. In order to avoid multiple requests, set `{endpoint}.{Class}()` to a variable. See sample code below.

## Today's Score Board


In [ ]:
# Query nba.live.endpoints.scoreboard and  list games in localTimeZone
from datetime import datetime, timezone
from dateutil import parser
from nba_api.live.nba.endpoints import scoreboard

f = "{gameId}: {awayTeam} vs. {homeTeam} @ {gameTimeLTZ}" 

board = scoreboard.ScoreBoard()
print("ScoreBoardDate: " + board.score_board_date)
games = board.games.get_dict()
for game in games:
    gameTimeLTZ = parser.parse(game["gameTimeUTC"]).replace(tzinfo=timezone.utc).astimezone(tz=None)
    print(f.format(gameId=game['gameId'], awayTeam=game['awayTeam']['teamName'], homeTeam=game['homeTeam']['teamName'], gameTimeLTZ=gameTimeLTZ))

## Box Score

In [ ]:
# Get BoxScore
from nba_api.live.nba.endpoints import boxscore
box = boxscore.BoxScore('0022400333') 

In [ ]:
from nba_api.live.nba.endpoints import boxscore
# Data Sets
last_score = ""
game_status = 0
import time
while game_status != 3:
    try:
        box = boxscore.BoxScore('0022400335') 
        info = box.game.get_dict()                    #equal to box.get_dict()['game']
        with open('boxscore.json', 'w') as f:
            f.write(box.get_json())
        score = f"{info['awayTeam']['score']} - {info['homeTeam']['score']}"
        if score != last_score:
            last_score = score
            print(f"team {info['awayTeam']['teamName']} vs. {info['homeTeam']['teamName']}")
            print(f"score: {score}")
        game_status = info['gameStatus']
    except Exception as e:
        print(f"Error {e}")

    time.sleep(1)

# with open('boxscore.json', 'w') as f:
#     f.write(box.get_json())
#box.arena.get_dict()                  #equal to box.get_dict()['game']['arena']
#box.away_team.get_dict()              #equal to box.get_dict()['game']['awayTeam']
#box.away_team_player_stats.get_dict() #equal to box.get_dict()['game']['awayTeam']['players']
#box.away_team_stats.get_dict()        #equal to box.get_dict()['game']['homeTeam'] w/o ['players']
#box.home_team.get_dict()              #equal to box.get_dict()['game']['homeTeam']
#box.home_team_player_stats.get_dict() #equal to box.get_dict()['game']['homeTeam']['players']
#box.home_team_stats.get_dict()        #equal to box.get_dict()['game']['homeTeam'] w/o ['players']
#box.game_details.get_dict()           #equal to box.get_dict()['game'] scrubbed of all other dictionaries
#box.officials.get_dict()              #equal to box.get_dict()['game']['officials']

In [ ]:
# Getting Box Scores. 
# Note: home_team & away_team have the identicial data structure.
players = box.away_team.get_dict()['players']
f = "{player_id}: {name}: {points} PTS"
for player in players:
    print(f.format(player_id=player['personId'],name=player['name'],points=player['statistics']['points']))

## Play By Play Data

In [ ]:
# Query nba.live.endpoints for the score board of GameID 0022000180 = NYK vs BOS
# Simple PlayByPlay Loop demonstrating data usage
from nba_api.live.nba.endpoints import playbyplay
from nba_api.stats.static import players

pbp = playbyplay.PlayByPlay('0022000196')
line = "{action_number}: {period}:{clock} {player_id} ({action_type})"
actions = pbp.get_dict()['game']['actions'] #plays are referred to in the live data as `actions`
for action in actions:
    player_name = ''
    player = players.find_player_by_id(action['personId'])
    if player is not None:
        player_name = player['full_name']
    print(line.format(action_number=action['actionNumber'],period=action['period'],clock=action['clock'],action_type=action['actionType'],player_id=player_name))